In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
pd.set_option("display.float_format", "{:.12g}".format)
np.set_printoptions(suppress=True, precision=12) 

In [2]:
def readfile(input_filename):

     
     # Open and parse the file
     with open(input_filename, "r") as file:
          fullrun=False
          for line in file:
               
               if line.startswith("=== EVENT 1 ==="):
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_starttime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_starttime)

               if line.startswith("=== EVENT 10000 ==="):
                    fullrun=True
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_endtime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_endtime)

     #print(fullrun)
     if (fullrun==False):
          print("run with less than 10000 events. Stopping the execution")
          run_endtime = "0h0m0s,0.0.0ns"
          run_starttime = "0h0m0s,0.0.0ns"   
                             

     return run_starttime, run_endtime


In [3]:
def parse_time(time_str):
    """Extract hours, minutes, seconds, and nanoseconds from a given time string."""
    match = re.match(r"(\d+)h(\d+)m(\d+)s,(\d+)\.(\d+)\.(\d+)ns", time_str)
    if match:
        hours, minutes, seconds, ns1, ns2, ns3 = map(int, match.groups())
        nanoseconds = (hours * 3600 + minutes * 60 + seconds) * 1_000_000_000 + ns1 * 1_000_000 + ns2 * 1_000 + ns3
        return nanoseconds
    else:
        raise ValueError("Invalid time format")

In [4]:
def format_time(nanoseconds):
    """Convert nanoseconds to a human-readable h:m:s,ns format."""
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours}h{minutes}m{seconds}s,{ns:09d}ns"

In [5]:
def format_time_seconds(nanoseconds):
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    return total_seconds


In [6]:
def extract_rate(nanoseconds,Nevts):
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    return Nevts/total_seconds

In [7]:
#define path to files
file_path="/Users/bordonis/ResearchActivities/Perovskite/data/"

In [12]:
def compute_rate(_filelist):
    #define output as a list of three items: [filename, run time duration (str), run time duration (sec) ,rate] 

    outputlist = []

    for f in _filelist:

        starttime, endtime = readfile(file_path+f)

        # Convert to nanoseconds
        start_ns = parse_time(starttime.strip())
        end_ns = parse_time(endtime.strip())

        # Compute difference
        diff_ns = end_ns - start_ns

        # Format the result
        time_diff_str = format_time(diff_ns)
        time_diff_sec = format_time_seconds(diff_ns)

        rate = extract_rate(diff_ns, 10000)

        outputlist.append((f, time_diff_str, time_diff_sec,rate))
        print("rate is [Hz]: ", rate )

    return outputlist

In [13]:
#CERN Cs104-A no source
filelist_nosource = ["newBox/Run_085_Data_5_22_2025_Ascii_Cs104.dat", #1mV - CERN display partial
            "newBox/Run_086_Data_5_22_2025_Ascii_Cs104.dat", #3mV - CERN display partial
            "newBox/Run_087_Data_5_22_2025_Ascii_Cs104.dat", #5mV - CERN display partial
            "newBox/Run_088_Data_5_22_2025_Ascii_Cs104.dat", #7mV - CERN display partial
            "newBox/Run_089_Data_5_22_2025_Ascii_Cs104.dat", #10mV - CERN display partial
            "newBox/Run_090_Data_5_22_2025_Ascii_Cs104.dat", #12mV - CERN display partial
            "newBox/Run_091_Data_5_22_2025_Ascii_Cs104.dat", #15mV - CERN display partial
            "newBox/Run_084_Data_5_21_2025_Ascii_CERN_Cs104-A.dat"]  #17mV - CERN display partial


#CERN Cs104-A Cs137
filelist_Cs137 = ["newBox/Run_092_Data_5_22_2025_Ascii_Cs104.dat", #10mV - CERN display partial
            "newBox/Run_093_Data_5_22_2025_Ascii_Cs104.dat", #12mV - CERN display partial
            "newBox/Run_094_Data_5_22_2025_Ascii_Cs104.dat", #15mV - CERN display partial
            "newBox/Run_095_Data_5_22_2025_Ascii_Cs104.dat", #17mV - CERN display partial
            "newBox/Run_096_Data_5_22_2025_Ascii_Cs104.dat"] #20mV - CERN display partial
            


#CERN Cs104-A Am241
filelist_Am241 = ["newBox/Run_097_Data_5_22_2025_Ascii_Cs104.dat", #10mV - CERN display partial
            "newBox/Run_098_Data_5_22_2025_Ascii_Cs104.dat", #12mV - CERN display partial
            "newBox/Run_099_Data_5_22_2025_Ascii_Cs104.dat", #15mV - CERN display partial
            "newBox/Run_100_Data_5_23_2025_Ascii_Cs104.dat", #17mV - CERN display partial
            "newBox/Run_101_Data_5_23_2025_Ascii_Cs104.dat"] #20mV - CERN display partial
            

In [14]:
print('computing rate for runs with no source')
nosourcelist = compute_rate(filelist_nosource)

print('\n computing rate for runs with Cs137 source')

Cs137list = compute_rate(filelist_Cs137)

print('\n computing rate for runs with Am241 source')

Am241list = compute_rate(filelist_Am241)

computing rate for runs with no source
rate is [Hz]:  909.0909090909091
rate is [Hz]:  833.3333333333334
rate is [Hz]:  909.0909090909091
rate is [Hz]:  909.0909090909091
rate is [Hz]:  322.5806451612903
rate is [Hz]:  102.04081632653062
rate is [Hz]:  13.793103448275861
rate is [Hz]:  2.9886431560071727

 computing rate for runs with Cs137 source
rate is [Hz]:  454.54545454545456
rate is [Hz]:  156.25
rate is [Hz]:  54.94505494505494
rate is [Hz]:  37.453183520599254
rate is [Hz]:  25.0

 computing rate for runs with Am241 source
rate is [Hz]:  303.030303030303
rate is [Hz]:  89.28571428571429
rate is [Hz]:  8.984725965858042
rate is [Hz]:  3.013863773357444
rate is [Hz]:  0.6674676278200508


In [21]:
pd.set_option('max_colwidth', 100)
pd.options.display.float_format = '{0:0.2f}'.format
#df_out = pd.DataFrame(outputlist, columns=['File', 'Time duration [str]', 'Run Time' , 'Rate'], index=[ "10mV", "12mV", "15mV", "17mV", "20mV"])
#df_out = pd.DataFrame(outputlist, columns=['File', 'Time duration [str]', 'Run Time' , '#Rate'], index=[ "UniGE - OFF", "UniGE - ON ", "CERN - OFF"])
df_nosource= pd.DataFrame(nosourcelist, columns=['File', 'Time duration [str]', 'Run Time' , 'Rate'], index=[ "1mV", "3mV", "5mV", "7mV", "10mV", "12mV", "15mV", "17mV"])

df_Cs137= pd.DataFrame(Cs137list, columns=['File', 'Time duration [str]', 'Run Time' , 'Rate'], index=[ "10mV", "12mV", "15mV", "17mV", "20mV"])

df_Am241= pd.DataFrame(Am241list, columns=['File', 'Time duration [str]', 'Run Time' , 'Rate'], index=[ "10mV", "12mV", "15mV", "17mV", "20mV"])

df_nosource.head(10)



,File,Time duration [str],Run Time,Rate
1mV,newBox/Run_085_Data_5_22_2025_Ascii_Cs104.dat,"0h0m11s,898414553ns",11,909.09
3mV,newBox/Run_086_Data_5_22_2025_Ascii_Cs104.dat,"0h0m12s,141076393ns",12,833.33
5mV,newBox/Run_087_Data_5_22_2025_Ascii_Cs104.dat,"0h0m11s,949696398ns",11,909.09
7mV,newBox/Run_088_Data_5_22_2025_Ascii_Cs104.dat,"0h0m11s,891599503ns",11,909.09
10mV,newBox/Run_089_Data_5_22_2025_Ascii_Cs104.dat,"0h0m31s,182579043ns",31,322.58
12mV,newBox/Run_090_Data_5_22_2025_Ascii_Cs104.dat,"0h1m38s,128484518ns",98,102.04
15mV,newBox/Run_091_Data_5_22_2025_Ascii_Cs104.dat,"0h12m5s,220303352ns",725,13.79
17mV,newBox/Run_084_Data_5_21_2025_Ascii_CERN_Cs104-A.dat,"0h55m46s,377167636ns",3346,2.99


In [22]:
df_Cs137.head(10)



,File,Time duration [str],Run Time,Rate
10mV,newBox/Run_092_Data_5_22_2025_Ascii_Cs104.dat,"0h0m22s,000342520ns",22,454.55
12mV,newBox/Run_093_Data_5_22_2025_Ascii_Cs104.dat,"0h1m4s,796600910ns",64,156.25
15mV,newBox/Run_094_Data_5_22_2025_Ascii_Cs104.dat,"0h3m2s,809382598ns",182,54.95
17mV,newBox/Run_095_Data_5_22_2025_Ascii_Cs104.dat,"0h4m27s,398657693ns",267,37.45
20mV,newBox/Run_096_Data_5_22_2025_Ascii_Cs104.dat,"0h6m40s,621565282ns",400,25.00


In [23]:
df_Am241.head(10)

,File,Time duration [str],Run Time,Rate
10mV,newBox/Run_097_Data_5_22_2025_Ascii_Cs104.dat,"0h0m33s,175277573ns",33,303.03
12mV,newBox/Run_098_Data_5_22_2025_Ascii_Cs104.dat,"0h1m52s,718383674ns",112,89.29
15mV,newBox/Run_099_Data_5_22_2025_Ascii_Cs104.dat,"0h18m33s,039794746ns",1113,8.98
17mV,newBox/Run_100_Data_5_23_2025_Ascii_Cs104.dat,"0h55m18s,707719863ns",3318,3.01
20mV,newBox/Run_101_Data_5_23_2025_Ascii_Cs104.dat,"4h9m42s,726620457ns",14982,0.67
